In [1]:
# %load startup
import numpy as np
from pathlib import Path
import time

from IPython.core.display import display, HTML
display(HTML("<style>"
             + "#notebook { padding-top:0px !important; } "
             + ".container { width:100% !important; } "
             + ".end_space { min-height:0px !important; } "
             + "</style>"))

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import pandas as pd
pd.options.display.max_colwidth = 100
pd.options.display.precision = 2

import pprint
pp = pprint.PrettyPrinter(indent=4, width=100)

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(module)s:%(lineno)d %(funcName)s %(levelname)s %(message)s')

In [5]:
from model_config import ModelConfig

def show_result(game_result_path):
    game_result_path = Path(game_result_path)
    for i, csv_file in enumerate(game_result_path.glob('*.csv')):
        game_number = csv_file.stem.split('_')[1]
        game_result_pd = pd.read_csv(csv_file)
        print(game_result_pd.describe())
        rewards = game_result_pd['reward'].values
        win_rate = np.sum(rewards>0)/len(rewards) 
        median_time = np.median(game_result_pd['time'].values)
        game_config_file = game_result_path/f'config_{game_number}.yaml'
        if game_config_file.exists():
            config = ModelConfig.from_yaml(game_config_file)
            print(f'{game_number}: black win rate={win_rate}, '
                  f'num_games={len(game_result_pd)}, '
                  f'time/game={median_time:.3f}, '
                  f'rollout={config.mcts_num_rollout}, '
                  f'komi={config.komi}')
        else:
            print(f'{game_number}: black win rate={win_rate}, '
                  f'num_games={len(game_result_pd)}, '
                  f'time/game={median_time:.3f}')

In [15]:
show_result('/home/peng_liu/src/tmp/game_result/random_vs_random/')

2: black win rate=0.766, num_games=1000, time/game=0.002, rollout=1000, komi=0
1: black win rate=0.45, num_games=100, time/game=0.002


In [17]:
show_result('/home/peng_liu/src/tmp/game_result/nn_vs_random/')

2: black win rate=0.763, num_games=1000, time/game=0.090, rollout=1000, komi=0
1: black win rate=0.157, num_games=1000, time/game=0.365


In [6]:
show_result('/home/peng_liu/src/tmp/game_result/mcts_vs_random/')

       Unnamed: 0     plys   reward    score     time
count     1000.00  1000.00  1000.00  1000.00  1000.00
mean       499.50    47.85     0.92    -7.08    20.13
std        288.82    30.63     0.40    10.83     8.64
min          0.00     0.00    -1.00   -81.00     1.37
25%        249.75    21.00     1.00   -10.00    13.43
50%        499.50    45.00     1.00    -3.00    22.14
75%        749.25    73.00     1.00    -1.00    27.47
max        999.00   135.00     1.00     6.00    36.50
2: black win rate=0.959, num_games=1000, time/game=22.138, rollout=1000, komi=0.0
       Unnamed: 0    plys  reward  score   time
count       10.00   10.00    10.0  10.00  10.00
mean         4.50  136.50     1.0 -48.20  24.91
std          3.03   39.71     0.0  25.95   4.70
min          0.00   67.00     1.0 -75.50  16.48
25%          2.25  122.75     1.0 -65.00  23.62
50%          4.50  148.50     1.0 -58.50  26.26
75%          6.75  162.00     1.0 -32.75  27.49
max          9.00  180.00     1.0  -2.50  30.51


In [29]:
action_space_size = 9 * 9 + 1
mcts_dirichlet_alpha = 1.0
legal_actions = [0, 2, 4]
prior_probabilities = np.zeros(action_space_size, dtype=float)
prior_probabilities[legal_actions] = np.random.dirichlet([mcts_dirichlet_alpha] * len(legal_actions))
print(prior_probabilities)

[0.09043658 0.         0.6326711  0.         0.27689232 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
